<a href="https://colab.research.google.com/github/droidadroit/age-and-gender-classification/blob/master/AgeGender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Mounting the drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Directory structure

content  
---- gdrive  
-------- My Drive  
------------ **AgeGenderClassification**  
---------------- **data**  
---------------- **models**  
---------------- **results**  
---------------- **AgeGender.ipynb**





## Installing PyTorch

In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

## Imports

In [0]:
import torch
import torch.autograd.variable as Variable
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as utils
from torch.utils.data import Dataset, DataLoader

In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [0]:
!pip install Pillow==5.3.0
from PIL import Image

# Preparing dataloaders

## **Raw data**



### Downloading data


We use the Adience dataset consisting unfiltered faces ([Link](http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/aligned.tar.gz)).  
Download it in the **data** directory. Then, we unzip it.

In [0]:
#!wget --user adiencedb --password adience http://www.cslab.openu.ac.il/download/adiencedb/AdienceBenchmarkOfUnfilteredFacesForGenderAndAgeClassification/aligned.tar.gz -P "/content/gdrive/My Drive/AgeGenderClassification/data"

In [0]:
#!tar xvzf "/content/gdrive/My Drive/AgeGenderClassification/data/aligned.tar.gz" -C "/content/gdrive/My Drive/AgeGenderClassification/data/"

### Downloading folds

All five folds used are present [here](https://github.com/GilLevi/AgeGenderDeepLearning/tree/master/Folds/train_val_txt_files_per_fold). Download the **train_val_txt_files_per_fold** folder and place it in the **data** directory.


## Data loading

In [0]:
PATH_TO_FOLDS = "/content/gdrive/My Drive/AgeGenderClassification/data/train_val_txt_files_per_fold"
PATH_TO_DATA = "/content/gdrive/My Drive/AgeGenderClassification/data"
PATH_TO_IMAGE_FOLDERS = PATH_TO_DATA + "/aligned"

### Creating a Dataset class

We create a class **`AdienceDataset`** that extends **`Dataset`**.

In [0]:
class AdienceDataset(Dataset):
    
    def __init__(self, txt_file, root_dir, transform):
        self.txt_file = txt_file
        self.root_dir = root_dir
        self.transform = transform
        self.data = self.read_from_txt_file()
    
    def __len__(self):
        return len(self.data)

    def read_from_txt_file(self):
        data = []
        f = open(self.txt_file)
        for line in f.readlines():
            image_file, label = line.split()
            label = int(label)
            if 'gender' in self.txt_file:
                label += 8
            data.append((image_file, label))
        return data
    
    def __getitem__(self, idx):
        img_name, label = self.data[idx]
        image = Image.open(self.root_dir + '/' + img_name)
        
        if self.transform:
            image = self.transform(image)
            
        return {
            'image': image,
            'label': label
        }         

### Transforms

In [0]:
transforms_list = [
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]

transforms_dict = {
    'train': {
        0: list(transforms_list[i] for i in [0, 1, 3]),
        1: list(transforms_list[i] for i in [0, 1, 2, 3])
    },
    'val': {
        0: list(transforms_list[i] for i in [0, 1, 3])
    },
    'test': {
        0: list(transforms_list[i] for i in [0, 1, 3])
    }
}

### Dataloader

In [0]:
def get_dataloader(s, c, fold, transform_index, minibatch_size):

    txt_file = f'{PATH_TO_FOLDS}/test_fold_is_{fold}/{c}_{s}.txt'
    root_dir = PATH_TO_IMAGE_FOLDERS
    
    transformed_dataset = AdienceDataset(txt_file, root_dir,
                                         transforms.Compose(transforms_dict[s][transform_index]))
    dataloader = DataLoader(transformed_dataset, batch_size=5, shuffle=False, num_workers=4)
    
    return dataloader

# Network

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
PATH_TO_MODELS = "/content/gdrive/My Drive/AgeGenderClassification/models"

## Defining the network

In [0]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # Conv layer 1
        self.conv1 = nn.Conv2d(3, 96, 7, stride = 4, padding = 1)
        self.pool1 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm1 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        # Conv layer 2
        self.conv2 = nn.Conv2d(96, 256, 5, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm2 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        # Conv layer 3
        self.conv3 = nn.Conv2d(256, 384, 3, stride = 1, padding = 1)
        self.pool3 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm3 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        # FC 1
        self.fc1 = nn.Linear(18816, 512)
        self.dropout1 = nn.Dropout(0.5)

        # FC 2
        self.fc2 = nn.Linear(512, 512)
        self.dropout2 = nn.Dropout(0.5)
  
        self.fc3 = nn.Linear(512, 10)
    
        self.apply(weights_init)

    
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = self.pool1(x)
        x = self.norm1(x)

        x = F.leaky_relu(self.conv2(x))
        x = self.pool2(x)
        x = self.norm2(x)
      
        x = F.leaky_relu(self.conv3(x))
        x = self.pool3(x)
        x = self.norm3(x)
      
        x = x.view(-1, 18816)
        
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dropout1(x)
      
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.dropout2(x)
      
        x = F.log_softmax(self.fc3(x))
  
        return x

In [0]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0, std=1e-2)

## Hyperparameters

In [0]:
minibatch_size = 5
num_epochs = 12
criterion = nn.NLLLoss()
lr = 0.0001

## Training the network

In [0]:
def train(net, train_dataloader, epochs, filename, checkpoint_frequency, val_dataloader=None):
    
    optimizer = optim.Adam(net.parameters(), lr)
    validation_loss = []
    checkpoint = 0
    iteration = 0
    
    for epoch in range(epochs):
        
        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            if i % 100 == 0:
                print(i, 'train')
                        
            if iteration % checkpoint_frequency == 0 and val_dataloader is not None:
                validation_loss.append(validate(net, val_dataloader))
                print(f'minibatch:{i}, epoch:{epoch}, iteration:{iteration}, validation_error:{validation_loss[-1]}')
                save_network(net, f'{filename}_checkpoint{checkpoint}')
                checkpoint += 1
            
            iteration += 1

    return net, validation_loss

## Validation

In [0]:
def validate(net, dataloader):
    total_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            total_loss += float(loss.item())
            if i % 100 == 0:
                print(i, 'validate')

    return total_loss/(i+1)

## Testing

In [0]:
def test(net, dataloader, c):
    result = {
        'exact_match': 0,
        'total': 0
    }
    if c == 'age':
        result['one_off_match'] = 0

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = net(images)
            outputs = torch.tensor(list(map(lambda x: torch.max(x, 0)[1], outputs))).to(device)
            result['total'] += len(outputs)
            result['exact_match'] += sum(outputs == labels).item()
            if c == 'age':
                result['one_off_match'] += (sum(outputs==labels) +
                                            sum(outputs==labels-1) +
                                            sum(outputs==labels+1)).item()
            if i % 100 == 0:
                print(i, 'test')

    return result           

## Saving the network

In [0]:
def save_network(net, filename):
    torch.save(net.state_dict(), f'{PATH_TO_MODELS}/{filename}.pt')

## Execution

### Train the network and save the models

In [0]:
def train_save(c, fold, train_transform_index, checkpoint_frequency=500):
    trained_net, validation_loss = train(
        Net().to(device),
        get_dataloader('train', c, fold, train_transform_index, minibatch_size),
        num_epochs,
        f'{fold}_{c}_train_{train_transform_index}',
        checkpoint_frequency,
        get_dataloader('val', c, fold, 0, minibatch_size)
    )
    
    plt.plot(list(map(lambda x: checkpoint_frequency * x, (list(range(0, len(validation_loss)))))), validation_loss)
    plt.xlabel('iterations')
    plt.ylabel('validation loss')
    plt.show()
    for index, l in enumerate(validation_loss):
        print(f'iteration:{index * checkpoint_frequency}, validation_loss={l}')
           
    return validation_loss

### Choose the best model

In [0]:
def choose_best_model(c, fold, train_transform_index, validation_loss):
    index = validation_loss.index(min(validation_loss))
    filename = f'{fold}_{c}_train_{train_transform_index}'
    for file in os.listdir(PATH_TO_MODELS):
        if file.startswith(filename):
            if file.startswith(f'{filename}_checkpoint{index}'):
                pass
            else:
                os.remove(f'{PATH_TO_MODELS}/{file}')

In [0]:
def get_best_model_filename(c, fold, train_transform_index):
    start_of_filename = f'{fold}_{c}_train_{train_transform_index}_checkpoint'
    for file in os.listdir(PATH_TO_MODELS):
        if file.startswith(start_of_filename):
            return file

### Get performance of a model

In [0]:
def get_performance(c, fold, train_transform_index):
    filename = get_best_model_filename(c, fold, train_transform_index)
    net = Net().to(device)
    net.load_state_dict(torch.load(f'{PATH_TO_MODELS}/{filename}'))
    performance = test(
        net,
        get_dataloader('test', c, fold, 0, minibatch_size),
        c
    )
    if c == 'age':
        return {
            'accuracy': performance['exact_match']/performance['total'],
            'one-off accuracy': performance['one_off_match']/performance['total']
        }
    elif c == 'gender':
        return {
            'accuracy': performance['exact_match']/performance['total']
        }

In [0]:
def get_validation_error(c, fold, train_transform_index):
    filename = get_best_model_filename(c, fold, train_transform_index)
    net = Net().to(device)
    net.load_state_dict(torch.load(f'{PATH_TO_MODELS}/{filename}'))
    return validate(net, get_dataloader('val', c, fold, 0, minibatch_size))

In [0]:
validation_loss = train_save(c='age', fold=3, train_transform_index=1)

0 train


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 validate
100 validate
200 validate
minibatch:0, epoch:0, iteration:0, validation_error:2.306431763792691
100 train
200 train
300 train
400 train
500 train
0 validate
100 validate
200 validate
minibatch:500, epoch:0, iteration:500, validation_error:1.9387054994498214
600 train
700 train
800 train
900 train
1000 train
0 validate
100 validate
200 validate
minibatch:1000, epoch:0, iteration:1000, validation_error:1.9742665009139335
1100 train
1200 train
1300 train
1400 train
1500 train
0 validate
100 validate
200 validate
minibatch:1500, epoch:0, iteration:1500, validation_error:1.931147381867448
1600 train
1700 train
1800 train
1900 train
2000 train
0 validate
100 validate
200 validate
minibatch:2000, epoch:0, iteration:2000, validation_error:2.077457299379453
2100 train
2200 train
2300 train
2400 train
2500 train
0 validate
100 validate
200 validate
minibatch:2500, epoch:0, iteration:2500, validation_error:1.8833135456255037
0 train
100 train
200 train
300 train
400 train
0 validate
10

In [0]:
choose_best_model('age', 3, 1, validation_loss)

In [32]:
performance = get_performance('age', 3, 1)
print(performance)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 test
100 test
200 test
300 test
400 test
500 test
600 test
{'accuracy': 0.5327942497753818, 'one-off accuracy': 0.8496555855046422}


In [28]:
error = get_validation_error('age', 3, 1)
print(error)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 validate
100 validate
200 validate
0.9154792693020268


In [0]:
# started at 17:43